In [1]:
import sentencepiece as spm
import pandas as pd
from tokenizers import SentencePieceBPETokenizer
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import SparseAdam
from transformers import (
    T5Model, 
    T5ForConditionalGeneration, 
    AdamW,
)
import pytorch_lightning as pl
import time
from datetime import datetime
import textwrap

device = torch.device(
    'cuda:0' if torch.cuda.is_available() else 'cpu'
)
print(f'device = {device}')

device = cuda:0


In [2]:
boDataForTokenizerPath = '../data/boTokenData.txt'
enDataForTokenizerPath = '../data/enTokenData.txt'

boDataPath = '../data/train.bo'
enDataPath = '../data/train.en'

boTokenizerPath = '../preProcessing/bo.model'
enTokenizerPath = '../preProcessing/en.model'

## Load data 


In [3]:
boFile = open(boDataPath, 'r', encoding = 'utf-8')
enFile = open(enDataPath, 'r', encoding = 'utf-8')

dataMatrix = []

while True: 
    boLine = boFile.readline().strip()
    enLine = enFile.readline().strip()
    if not boLine or not enLine: 
        break 
    dataMatrix.append([boLine, enLine])
  
# Create pandas dataframe 
df = pd.DataFrame(dataMatrix, columns = ['bo', 'en'])
df

,bo,en
0,རྒྱལ་པོ་ཞེས་བྱ་བས་རྒྱལ་སྲིད་འབྱོར་པ་རྒྱས་པ་བདེ...,under his rule the kingdom prospered and thriv...
1,དེས་དཔུང་གི་ཚོགས་ཡན་ལག་བཞི་པ་གླང་པོ་ཆེ་པའི་ཚོག...,he called up the four branches of his armed fo...
2,སུམ་ཅུ་རྩ་གསུམ་པའི་ལྷ་རྣམས་ཀྱི་ཁ་དོག་གི་མཐུ་བས...,bathed in a vast light more luminous than the ...
3,མ་མ་བརྒྱད་པོ་པང་ན་འཚོ་བའི་མ་མ་གཉིས་དང་ནུ་མ་སྣུ...,was entrusted to eight nursemaids two to cuddl...
4,རྒྱལ་པོ་རྒྱལ་རིགས་སྤྱི་བོར་དབང་བསྐུར་བ་ལྗོངས་ཀ...,he trained in and mastered those arts and skil...
...,...,...
106861,མད་གལ་གྱི་བུ་དེ་བཞིན་གཤེགས་པ་དགྲ་བཅོམ་པ་ཡང་དག་...,maudgalyayana the thusgone worthy perfect budd...
106862,བཅོམ་ལྡན་འདས་ཀྱིས་དེ་སྐད་ཅེས་བཀའ་སྩལ་པ་དང་་ཚེ་...,when the blessed one had spoken venerable maha...
106863,འཕགས་པ་བཅོམ་ལྡན་འདས་ཀྱི་ཡེ་ཤེས་རྒྱས་པའི་མདོ་སྡ...,this completes the great vehicle sutra the pre...
106864,རྒྱ་གར་གྱི་མཁན་པོ་པྲཛྙ་བར་མ་དང་་ལོཙྪ་བ་བན་དེ་ཡ...,this was translated by the indian preceptor pr...


In [14]:
boTextsAll = df['bo'].tolist()
enTextsAll = df['en'].tolist()

## Tokenizers for Tibetan and English

The code cell below uses Google SentencePiece tokenizer, but we cannot yet figure out how to truncate and pad the tokenizations to the same length, nor the special characters. Save the code but we will not use it for now. 

In [5]:
'''
## Ignore this cell
'''

# Load tokenizers that are already trained
boTokenizer = spm.SentencePieceProcessor(model_file=boTokenizerPath)
enTokenizer = spm.SentencePieceProcessor(model_file=enTokenizerPath)

# Verify for Tibetan
print(boTokenizer.encode(['ངའི་མིང་ལ་བསྟན་སྒྲོལ་མ་ཟེར་'], out_type=str))
print(boTokenizer.encode(['ངའི་མིང་ལ་བསྟན་སྒྲོལ་མ་ཟེར་', 'བཀ྄ྲ་ཤིས་བདེ་ལེགས།'], out_type=int))
print(boTokenizer.decode([4149, 306, 6, 245, 4660, 748]))
print(boTokenizer.decode(['▁ངའི་', 'མིང་', 'ལ་', 'བསྟན་', 'སྒྲོལ་མ་', 'ཟེར་']))
print('Vocab size of Tibetan Tokenizer:', boTokenizer.get_piece_size())

# Verify for English
print(enTokenizer.encode(["My name isn't Tenzin Dolma Gyalpo"], out_type=str))
print(enTokenizer.encode(['My name is Tenzin Dolma Gyalpo', 'Hello'], out_type=int))
print(enTokenizer.decode([[8803, 180, 12, 5519, 15171, 17894], [887, 21491]]))
print('Vocab size of English Tokenizer:', enTokenizer.get_piece_size())

[['▁ངའི་', 'མིང་ལ་', 'བསྟན་', 'སྒྲོལ་མ་', 'ཟེར་']]
[[3644, 18002, 530, 6257, 2154], [4, 3333, 0, 6081, 3, 6750, 1030, 2261, 1961, 0]]
ཆོས་སྟོན་ཏོ་་རང་གི་ལ་ཡོད་པའི་ ཨིན་ཡུལ་དང་ལྡན་པའི་
ངའི་མིང་ལ་བསྟན་སྒྲོལ་མ་ཟེར་
Vocab size of Tibetan Tokenizer: 32000
[['▁My', '▁name', '▁is', 'n', "'", 't', '▁Tenzin', '▁Dolma', '▁Gyalpo']]
[[8803, 180, 12, 5519, 15171, 17894], [887, 21491]]
['My name is Tenzin Dolma Gyalpo', 'Hello']
Vocab size of English Tokenizer: 25000


Instead, we use huggingface tokenizer now. The bad news is we can no longer find the right API. 

In [4]:
boTokenizer = SentencePieceBPETokenizer()
boTokenizer.train([boDataForTokenizerPath], vocab_size = 32000, special_tokens = ['<s>', '<pad>', '</s>', '<unk>', '<mask>'])

enTokenizer = SentencePieceBPETokenizer()
enTokenizer.train([enDataForTokenizerPath], vocab_size = 25000, special_tokens = ['<s>', '<pad>', '</s>', '<unk>', '<mask>'])

print('Tibetan tokenizer vocab size:', boTokenizer.get_vocab_size())
print('English tokenizer vocab size:', enTokenizer.get_vocab_size())

Tibetan tokenizer vocab size: 32000
English tokenizer vocab size: 25000


In [5]:
# Verify for Tibetan
outputs = boTokenizer.encode_batch(['ངའི་མིང་ལ་བསྟན་སྒྲོལ་མ་ཟེར་', 'ངའི་མིང་ལ་བསྟན་སྒྲོལ་མ་ཟེར་', 'བཀ྄ྲ་ཤིས་བདེ་ལེགས།'])

for output in outputs: 
    print('ids:', output.ids)
    print('token:', output.tokens)
    print('mask:', output.attention_mask)

ids: [5608, 816, 9506, 3748, 17603]
token: ['▁ངའི་', 'མིང་', 'ལ་བསྟན་', 'སྒྲོལ་', 'མ་ཟེར་']
mask: [1, 1, 1, 1, 1]
ids: [5608, 816, 9506, 3748, 17603]
token: ['▁ངའི་', 'མིང་', 'ལ་བསྟན་', 'སྒྲོལ་', 'མ་ཟེར་']
mask: [1, 1, 1, 1, 1]
ids: [8557, 890, 1675, 148]
token: ['▁བཀྲ་ཤིས་', 'བདེ་', 'ལེ', 'གས']
mask: [1, 1, 1, 1]


In [6]:
# Verify for English
outputs = enTokenizer.encode_batch(["My name isn't Tenzin Dolma Gyalpo", 'My name is Tenzin Dolma Gyalpo', 'Hello'])

for output in outputs: 
    print('ids:', output.ids)
    print('token:', output.tokens)
    print('mask:', output.attention_mask)

ids: [16272, 1226, 168, 78, 12, 84, 13164, 22220, 225, 9256, 3135]
token: ['▁My', '▁name', '▁is', 'n', "'", 't', '▁Tenzin', '▁Dol', 'ma', '▁Gyal', 'po']
mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
ids: [16272, 1226, 168, 13164, 22220, 225, 9256, 3135]
token: ['▁My', '▁name', '▁is', '▁Tenzin', '▁Dol', 'ma', '▁Gyal', 'po']
mask: [1, 1, 1, 1, 1, 1, 1, 1]
ids: [636, 1345, 79]
token: ['▁H', 'ell', 'o']
mask: [1, 1, 1]


## Pytorch `Dataset`

In [7]:
class MyDataset(Dataset): 
    def __init__(self, boTexts, enTexts, boTokenizer, enTokenizer, boMaxLen, enMaxLen): 
        super().__init__()
        self.boTexts = boTexts
        self.enTexts = enTexts
        self.boTokenizer = boTokenizer
        self.enTokenizer = enTokenizer
        
        # Enable padding and truncation 
        self.boTokenizer.enable_padding(length = boMaxLen)
        self.boTokenizer.enable_truncation(max_length = boMaxLen)
        self.enTokenizer.enable_padding(length = enMaxLen)
        self.enTokenizer.enable_truncation(max_length = enMaxLen)
        
    ''' Return the size of dataset '''
    def __len__(self): 
        return len(self.boTexts)
    
    '''
    -- The routine for querying one data entry 
    -- The index of must be specified as an argument
    -- Return a dictionary 
    '''
    def __getitem__(self, idx): 
        # Apply tokenizer
        boOutputs = self.boTokenizer.encode(self.boTexts[idx])
        enOutputs = self.enTokenizer.encode(self.enTexts[idx])
        
        # Get numerical tokens 
        boEncoding = boOutputs.ids
        enEncoding = enOutputs.ids
        
        # Get attention mask 
        boMask = boOutputs.attention_mask
        enMask = enOutputs.attention_mask
        
        return {
            'source_ids': torch.tensor(boEncoding), 
            'source_mask': torch.tensor(boMask), 
            'target_ids': torch.tensor(enEncoding), 
            'target_mask': torch.tensor(enMask)
        }

## Define model class

In [9]:
class T5FineTuner(pl.LightningModule): 
    ''' Part 1: Define the architecture of model in init '''
    def __init__(self, hparams):
        super(T5FineTuner, self).__init__()
        self.model = T5ForConditionalGeneration.from_pretrained(
            hparams['pretrainedModelName'], 
            return_dict = True    # I set return_dict true so that outputs  are presented as dictionaries
        )
        self.boTokenizer = hparams['boTokenizer']
        self.enTokenizer = hparams['enTokenizer']
        self.hparams = hparams
        
        
    ''' Part 2: Define the forward propagation '''
    def forward(self, input_ids, attention_mask = None, decoder_input_ids = None, decoder_attention_mask = None, labels = None):  
        return self.model(
            input_ids, 
            attention_mask = attention_mask, 
            decoder_input_ids = decoder_input_ids, 
            decoder_attention_mask = decoder_attention_mask, 
            labels = labels
        )
    
    
    ''' Part 3: Configure optimizer and scheduler '''
    def configure_optimizers(self): 
        # I have no idea why to configure parameter this way 
        optimizer_grouped_parameters = [
            {
                # parameter with weight decay 
                'params': [param for name, param in model.named_parameters() if ('bias' not in name and 'LayerNorm.weight' not in name)], 
                'weight_decay': self.hparams['weight_decay'], 
            }, 
            {
                'params': [param for name, param in model.named_parameters() if ('bias' in name or 'LayerNorm.weight' in name)], 
                'weight_decay': 0.0, 
            }
        ]
        
        optimizer = AdamW(optimizer_grouped_parameters, lr = self.hparams['learning_rate'])
        return optimizer
    
    
    ''' Part 4.1: Training logic '''
    def training_step(self, batch, batch_idx):         
        loss = self._step(batch)
        self.log('train_loss', loss)
        return loss 
    
    
    def _step(self, batch): 
        labels = batch['target_ids'] 
        labels[labels[:, ] == 0] = -100    # Change the pad id from 0 to -100, but I do not know why the example chooses to do so. I will comment it out for now
        
        outputs = self(
            input_ids = batch['source_ids'], 
            attention_mask = batch['source_mask'], 
            labels = labels, 
            decoder_attention_mask = batch['target_mask']
        )
        
        return outputs.loss

    
    ''' Part 4.2: Validation logic '''
    def validation_step(self, batch, batch_idx):        
        loss = self._step(batch)
        self.log('val_loss', loss)
        
        
    ''' Part 4.3: Test logic '''
    def test_step(self, batch, batch_idx): 
        loss = self._step(batch)
        self.log('test_loss', loss)
    
    
    ''' Part 5: Data loaders '''
    def _get_dataloader(self, start_idx, end_idx): 
        dataset = MyDataset(
            boTexts = boTextsAll[start_idx:end_idx], 
            enTexts = enTextsAll[start_idx:end_idx], 
            boTokenizer = self.hparams['boTokenizer'], 
            enTokenizer = self.hparams['enTokenizer'], 
            boMaxLen = self.hparams['max_input_len'], 
            enMaxLen = self.hparams['max_output_len']
        )
        
        return DataLoader(dataset, batch_size = hparams['batch_size'])
    
    
    def train_dataloader(self): 
        start_idx = 0
        end_idx = int(self.hparams['train_percentage'] * len(boTextsAll))
        return self._get_dataloader(start_idx, end_idx)
        
    
    def val_dataloader(self): 
        start_idx = int(self.hparams['train_percentage'] * len(boTextsAll))
        end_idx = int((self.hparams['train_percentage'] + self.hparams['val_percentage']) * len(boTextsAll))
        return self._get_dataloader(start_idx, end_idx)
    
    
    def test_dataloader(self): 
        start_idx = int((self.hparams['train_percentage'] + self.hparams['val_percentage']) * len(boTextsAll))
        end_idx = len(boTextsAll)
        return self._get_dataloader(start_idx, end_idx)

In [10]:
hparams = {
    'boTokenizer': boTokenizer,
    'enTokenizer': enTokenizer,
    'pretrainedModelName': 't5-small', 
    'train_percentage': 0.85, 
    'val_percentage': 0.13, 
    'learning_rate': 3e-4, 
    'max_input_len': 100, 
    'max_output_len': 100, 
    'batch_size': 8, 
    'num_train_epochs': 2, 
    'num_gpu': 1, 
    'weight_decay': 0, 
}

## Training

In [12]:
torch.cuda.empty_cache()

train_params = dict(
    gpus = hparams['num_gpu'], 
    max_epochs = hparams['num_train_epochs'], 
    progress_bar_refresh_rate = 20, 
)

model = T5FineTuner(hparams)

trainer = pl.Trainer(**train_params)

trainer.fit(model)

# Save model for later use
now = datetime.now()
trainer.save_checkpoint('01_t5simple_bo_en' + now.strftime("%Y-%d-%m-%Y--%H=%M=%S") + '.ckpt')

trainer.test()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 60 M  


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': tensor(5.6364, device='cuda:0'),
 'train_loss': tensor(5.1073, device='cuda:0'),
 'val_loss': tensor(5.6757, device='cuda:0')}
--------------------------------------------------------------------------------



[{'train_loss': 5.107315540313721,
  'val_loss': 5.675739288330078,
  'test_loss': 5.636357307434082}]

## Testing

In [12]:
# Load a previously saved model

torch.cuda.empty_cache()

modelLoaded = T5FineTuner.load_from_checkpoint(checkpoint_path='__01_t5simple_bo_en_2020-07-12-2020--15=37=10.ckpt').to(device)

In [19]:
start_idx = 0
end_idx = 8

testset = MyDataset(
    boTexts = boTextsAll[start_idx:end_idx], 
    enTexts = enTextsAll[start_idx:end_idx], 
    boTokenizer = hparams['boTokenizer'], 
    enTokenizer = hparams['enTokenizer'], 
    boMaxLen = hparams['max_input_len'], 
    enMaxLen = hparams['max_output_len']
)

test_dataloader = DataLoader(testset, batch_size = hparams['batch_size'])
testit = iter(test_dataloader)

# Take one batch from testset 
batch = next(testit)

# Generate target ids
outs = modelLoaded.model.generate(
    batch['source_ids'].cuda(), 
    attention_mask = batch['source_mask'].cuda(), 
    use_cache = True, 
    decoder_attention_mask = batch['target_mask'], 
    max_length = hparams['max_output_len'], 
    num_beams = 5, 
    repetition_penalty = 2.5, 
    length_penalty = 1.0, 
    # early_stopping = True, 
)

pred_texts = [enTokenizer.decode(ids) for ids in outs.tolist()]
source_texts = [boTokenizer.decode(ids) for ids in batch['source_ids'].tolist()]
target_texts = [enTokenizer.decode(ids) for ids in batch['target_ids'].tolist()]

for i in range(len(pred_texts)): 
    lines = textwrap.wrap("Tibetan Text:\n%s\n" % source_texts[i], width=100)
    print("\n".join(lines))
    print("\nActual translation: %s" % target_texts[i])
    print("\nPredicted translation: %s" % pred_texts[i])
    print('=' * 50 + '\n')

Tibetan Text: རྒྱལ་པོ་ཞེས་བྱ་བས་རྒྱལ་སྲིད་འབྱོར་པ་རྒྱས་པ་བདེ་བ་ལོ་ལེགས་པ་སྐྱེ་བོ་དང་མི་མང་པོས་གང་བ་བ
ྱེད་དུ་བཅུག་གོ་

Actual translation: under his rule the kingdom prospered and thrived crops were bountiful and the land teemed with animals and people

Predicted translation: when the blessed one was in order to see the bodhisattva great beings who are born in this world with its gods and humans that is a gateway to the light of the dharma for the sake of awakening you will not be able to go forth as well as he has been taught by the dharma teaching of the dharma from his seat at the time of his death and so on up the thusgone one the perfect buddha known as an end to the four meditative states and the lord of the dharma or the three worlds

Tibetan Text: དེས་དཔུང་གི་ཚོགས་ཡན་ལག་བཞི་པ་གླང་པོ་ཆེ་པའི་ཚོགས་དང་རྟ་པའི་ཚོགས་དང་ཤིང་རྟ་པའི་ཚོགས་དང་ད
པུང་བུ་ཆུང་གི་ཚོགས་གོ་བསྐོན་ཏེ་ཡུལ་མ་ག་དཧའི་རྒྱལ་པོའི་ཁ་བ་མ་གཏོགས་པ་བཅོམ་ནས་ཕྱིར་ལྡོག་པར་བྱེད་དོ་

Actual translation: he called up the four branche

In [17]:
# %tensorboard --logdir lightning_logs/